# Application: Prétraitement du jeu de données NSL-KDD pour Deep Learning

Dans ce notebook, nous allons prétraiter ce jeu de données pour des tâches de classification par apprentissage profond.

## 1. Importation des bibliothèques nécessaires

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Pour afficher les graphiques dans le notebook
%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Chargement des données

Le NSL-KDD est disponible à l'URL: https://www.unb.ca/cic/datasets/nsl.html

Le jeu de données comporte deux fichiers principaux:
- KDDTrain+.txt: Ensemble d'entraînement
- KDDTest+.txt: Ensemble de test

In [62]:
# Définition des noms de colonnes pour le jeu de données NSL-KDD
col_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'attack_type', 'difficulty_level'
]

# Chargement des données
try:
    # Ajustez les chemins selon l'emplacement de vos fichiers
    train_data = pd.read_csv('KDDTrain+.txt', header=None, names=col_names)
    test_data = pd.read_csv('KDDTest+.txt', header=None, names=col_names)
    
    print(f"Forme des données d'entraînement: {train_data.shape}")
    print(f"Forme des données de test: {test_data.shape}")
    
except FileNotFoundError:
    print("Erreur: Les fichiers de données n'ont pas été trouvés.")
    print("Veuillez télécharger les fichiers depuis https://www.unb.ca/cic/datasets/nsl.html")
    print("et les placer dans le même répertoire que ce notebook.")

Forme des données d'entraînement: (125973, 43)
Forme des données de test: (22544, 43)


## 3. Exploration des données

Examinons brièvement les données pour comprendre leur structure.

In [63]:
# Affichage des premières lignes
train_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [64]:
# Statistiques descriptives
train_data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,serror_rate,...,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,difficulty_level
count,125973.000000,125973.0,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,0.004096,0.0,0.000008,0.009423,84.107555,27.737888,0.284485,0.282485,0.119958,0.121183,...,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240,19.504060
std,0.099370,0.0,0.002817,0.096612,114.508607,72.635840,0.446456,0.447022,0.320436,0.323647,...,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459,2.291503
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.0,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
50%,0.000000,0.0,0.000000,0.000000,14.000000,8.000000,0.000000,0.000000,0.000000,0.000000,...,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,0.000000,0.0,0.000000,0.000000,143.000000,18.000000,1.000000,1.000000,0.000000,0.000000,...,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,21.000000
max,9.000000,0.0,1.000000,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000


In [65]:
# Vérification des types d'attaques
print("Types d'attaques dans l'ensemble d'entraînement:")
print(train_data['attack_type'].value_counts())

print("\nTypes d'attaques dans l'ensemble de test:")
print(test_data['attack_type'].value_counts())

Types d'attaques dans l'ensemble d'entraînement:
attack_type
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64

Types d'attaques dans l'ensemble de test:
attack_type
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint              

## 4. Regroupement des types d'attaques

Pour simplifier la classification, nous allons regrouper les types d'attaques en 5 catégories principales:
1. Normal (trafic normal)
2. DoS (Denial of Service)
3. Probe (Reconnaissance)
4. R2L (Remote to Local)
5. U2R (User to Root)

In [66]:
# Dictionnaire de correspondance entre types d'attaques spécifiques et catégories principales
attack_mapping = {
    'normal': 'Normal',
    
    # DoS attacks
    'neptune': 'DoS', 'back': 'DoS', 'land': 'DoS', 'pod': 'DoS', 
    'smurf': 'DoS', 'teardrop': 'DoS', 'apache2': 'DoS', 'udpstorm': 'DoS',
    'processtable': 'DoS', 'worm': 'DoS', 'mailbomb': 'DoS',
    
    # Probe attacks
    'satan': 'Probe', 'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe',
    'mscan': 'Probe', 'saint': 'Probe',
    
    # R2L attacks
    'guess_passwd': 'R2L', 'ftp_write': 'R2L', 'imap': 'R2L', 'phf': 'R2L',
    'multihop': 'R2L', 'warezmaster': 'R2L', 'warezclient': 'R2L', 'spy': 'R2L',
    'xlock': 'R2L', 'xsnoop': 'R2L', 'snmpguess': 'R2L', 'snmpgetattack': 'R2L',
    'httptunnel': 'R2L', 'sendmail': 'R2L', 'named': 'R2L',
    
    # U2R attacks
    'buffer_overflow': 'U2R', 'loadmodule': 'U2R', 'rootkit': 'U2R', 'perl': 'U2R',
    'sqlattack': 'U2R', 'xterm': 'U2R', 'ps': 'U2R'
}

# Application du mapping aux ensembles d'entraînement et de test
train_data['attack_category'] = train_data['attack_type'].map(attack_mapping)
test_data['attack_category'] = test_data['attack_type'].map(attack_mapping)

# Vérification des catégories d'attaques regroupées
print("Distribution des catégories d'attaques dans l'ensemble d'entraînement:")
print(train_data['attack_category'].value_counts())

print("\nDistribution des catégories d'attaques dans l'ensemble de test:")
print(test_data['attack_category'].value_counts())

Distribution des catégories d'attaques dans l'ensemble d'entraînement:
attack_category
Normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: count, dtype: int64

Distribution des catégories d'attaques dans l'ensemble de test:
attack_category
Normal    9711
DoS       7460
R2L       2885
Probe     2421
U2R         67
Name: count, dtype: int64


## 5. Vérification et traitement des valeurs manquantes

In [67]:
# Vérification des valeurs manquantes dans l'ensemble d'entraînement
print("Valeurs manquantes dans l'ensemble d'entraînement:")
print(train_data.isnull().sum().sum())

# Vérification des valeurs manquantes dans l'ensemble de test
print("\nValeurs manquantes dans l'ensemble de test:")
print(test_data.isnull().sum().sum())

Valeurs manquantes dans l'ensemble d'entraînement:
0

Valeurs manquantes dans l'ensemble de test:
0


## 6. Encodage des variables catégorielles

In [68]:
# Identification des colonnes catégorielles (hors cible)
categorical_cols = ['protocol_type', 'service', 'flag']

# Préparation des transformateurs pour les colonnes catégorielles
categorical_preprocessors = {}

# Encodage des colonnes catégorielles avec OneHotEncoder
for col in categorical_cols:
    # Utilisation d'un encodeur pour chaque colonne catégorielle
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Ajustement de l'encodeur sur les données d'entraînement et de test combinées
    # pour s'assurer que toutes les catégories possibles sont encodées
    combined_data = pd.concat([train_data[col], test_data[col]], axis=0).values.reshape(-1, 1)
    encoder.fit(combined_data)
    
    # Stockage de l'encodeur pour une utilisation ultérieure
    categorical_preprocessors[col] = encoder

In [69]:
def preprocess_dataset(data, categorical_preprocessors, is_training=True):
    # Copie des données pour éviter de modifier l'original
    processed_data = data.copy()
    
    # Encodage one-hot pour chaque colonne catégorielle
    for col, encoder in categorical_preprocessors.items():
        # Transformation des données - conversion en array NumPy pour maintenir la cohérence
        encoded_array = encoder.transform(processed_data[col].values.reshape(-1, 1))
        
        # Création d'un DataFrame avec les données encodées
        feature_names = [f"{col}_{category}" for category in encoder.categories_[0]]
        encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=processed_data.index)
        
        # Concaténation avec le DataFrame principal
        processed_data = pd.concat([processed_data, encoded_df], axis=1)
        
        # Suppression de la colonne originale
        processed_data.drop(col, axis=1, inplace=True)
    
    # Suppression des colonnes non nécessaires
    processed_data.drop(['attack_type', 'difficulty_level'], axis=1, inplace=True)
    
    return processed_data

In [70]:
# Prétraitement des ensembles d'entraînement et de test
processed_train = preprocess_dataset(train_data, categorical_preprocessors, is_training=True)
processed_test = preprocess_dataset(test_data, categorical_preprocessors, is_training=False)

# Vérification des dimensions
print(f"Dimensions de l'ensemble d'entraînement prétraité: {processed_train.shape}")
print(f"Dimensions de l'ensemble de test prétraité: {processed_test.shape}")

Dimensions de l'ensemble d'entraînement prétraité: (125973, 26)
Dimensions de l'ensemble de test prétraité: (22544, 26)


## 7. Encodage de la variable cible

In [ ]:
# Encodage de la variable cible
label_encoder = LabelEncoder()
# Fit sur les données combinées pour s'assurer que toutes les classes sont encodées
combined_labels = pd.concat([processed_train['attack_category'], processed_test['attack_category']])
label_encoder.fit(combined_labels)

# Application de l'encodage
processed_train['attack_encoded'] = label_encoder.transform(processed_train['attack_category'])
processed_test['attack_encoded'] = label_encoder.transform(processed_test['attack_category'])

# Affichage de la correspondance entre les catégories et leurs encodages
for i, category in enumerate(label_encoder.classes_):
    print(f"{category}: {i}")

# Suppression de la colonne catégorielle originale
processed_train.drop('attack_category', axis=1, inplace=True)
processed_test.drop('attack_category', axis=1, inplace=True)

DoS: 0
Normal: 1
Probe: 2
R2L: 3
U2R: 4


## 8. Normalisation des caractéristiques numériques

In [ ]:
# Identification des colonnes numériques (excluant la cible encodée)
numeric_cols = processed_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols.remove('attack_encoded')  # Retirer la colonne cible

# Initialisation du scaler
scaler = StandardScaler()

# Ajustement du scaler sur les données d'entraînement
scaler.fit(processed_train[numeric_cols])

# Normalisation des colonnes numériques dans l'ensemble d'entraînement
processed_train[numeric_cols] = scaler.transform(processed_train[numeric_cols])

# Normalisation des colonnes numériques dans l'ensemble de test
processed_test[numeric_cols] = scaler.transform(processed_test[numeric_cols])

# Vérification des statistiques après normalisation
print("Statistiques des caractéristiques numériques après normalisation (ensemble d'entraînement):")
print(processed_train[numeric_cols].describe().T[['mean', 'std', 'min', 'max']])

Statistiques des caractéristiques numériques après normalisation (ensemble d'entraînement):
                                  mean       std         min         max
duration                 -5.302011e-18  1.000004   -0.041221   90.530138
src_bytes                -3.948306e-17  1.000004   -0.734511    3.728053
dst_bytes                 4.162642e-17  1.000004   -0.381878    6.653245
logged_in                 4.484147e-18  1.000004   -0.637209    1.602664
count                    -1.232435e-16  1.000004   -0.631929    1.605104
srv_count                 3.829857e-17  1.000004   -0.374362    2.746403
serror_rate              -1.847243e-17  1.000004   -0.374432    2.715365
srv_serror_rate          -1.618805e-16  1.000004   -1.503403    0.771283
rerror_rate              -1.714693e-17  1.000004   -0.349683    5.196208
srv_rerror_rate          -2.729971e-17  1.000004   -0.374560    3.474118
same_srv_rate            -1.756996e-17  1.000004   -1.836071    0.734343
diff_srv_rate            -5.8152

## 9. Préparation des données pour le deep learning

In [ ]:
# Séparation des caractéristiques (X) et de la cible (y)
X_train = processed_train.drop('attack_encoded', axis=1)
y_train = processed_train['attack_encoded']

X_test = processed_test.drop('attack_encoded', axis=1)
y_test = processed_test['attack_encoded']

# Nombre de classes
num_classes = y_train.nunique()
print(f"Nombre de classes: {num_classes}")

# Conversion en encodage one-hot pour le deep learning
y_train_onehot = to_categorical(y_train, num_classes=num_classes)
y_test_onehot = to_categorical(y_test, num_classes=num_classes)

# Conversion en tenseurs pour TensorFlow
X_train_tensor = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_onehot, dtype=tf.float32)

X_test_tensor = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_onehot, dtype=tf.float32)

Nombre de classes: 5


## 10. Création de datasets avec batching et shuffling

In [ ]:
# Paramètres
batch_size = 32
buffer_size = 1000  # Pour le shuffling

# Vérification des dimensions
print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
print(f"X_test_tensor shape: {X_test_tensor.shape}")
print(f"y_test_tensor shape: {y_test_tensor.shape}")

# Création du dataset d'entraînement avec drop_remainder=True
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# Création du dataset de test avec drop_remainder=True
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tensor, y_test_tensor))
test_dataset = test_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# Vérification
for features, labels in train_dataset.take(1):
    print(f"Forme des caractéristiques: {features.shape}")
    print(f"Forme des étiquettes: {labels.shape}")

X_train_tensor shape: (125973, 25)
y_train_tensor shape: (125973, 5)
X_test_tensor shape: (22544, 25)
y_test_tensor shape: (22544, 5)
Forme des caractéristiques: (32, 25)
Forme des étiquettes: (32, 5)


## 11. Démonstration d'un modèle de deep learning simple

In [ ]:
# Définition d'un modèle simple pour la démonstration
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        # input_shape doit être un tuple, pas un entier unique
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Création du modèle
input_shape = X_train.shape[1]  # Nombre de caractéristiques
model = create_model(input_shape, num_classes)

# Résumé du modèle
model.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,725 (57.52 KB)

 Trainable params: 14,277 (55.77 KB)

 Non-trainable params: 448 (1.75 KB)

## 12. Entraînement du modèle (exemple rapide)

Note: Pour un véritable entraînement, vous devriez utiliser plus d'époques et ajuster les hyperparamètres.

In [ ]:
# Définition des callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)
]

# Entraînement du modèle (avec seulement quelques époques pour démonstration)
history = model.fit(
    train_dataset,
    epochs=5,  # Un petit nombre d'époques pour la démonstration
    validation_data=test_dataset,
    callbacks=callbacks
)

Epoch 1/5
3936/3936 ━━━━━━━━━━━━━━━━━━━━ 4s 790us/step - accuracy: 0.9206 - loss: 0.2685 - val_accuracy: 0.7151 - val_loss: 1.4894 - learning_rate: 0.0010
Epoch 2/5
3936/3936 ━━━━━━━━━━━━━━━━━━━━ 3s 778us/step - accuracy: 0.9634 - loss: 0.1146 - val_accuracy: 0.7294 - val_loss: 1.5039 - learning_rate: 0.0010
Epoch 3/5
3936/3936 ━━━━━━━━━━━━━━━━━━━━ 3s 834us/step - accuracy: 0.9686 - loss: 0.0979 - val_accuracy: 0.7282 - val_loss: 1.5685 - learning_rate: 0.0010
Epoch 4/5
3936/3936 ━━━━━━━━━━━━━━━━━━━━ 3s 799us/step - accuracy: 0.9700 - loss: 0.0900 - val_accuracy: 0.7217 - val_loss: 1.9208 - learning_rate: 0.0010
Epoch 5/5
3936/3936 ━━━━━━━━━━━━━━━━━━━━ 3s 782us/step - accuracy: 0.9733 - loss: 0.0804 - val_accuracy: 0.7351 - val_loss: 1.7979 - learning_rate: 1.0000e-04


## 13. Évaluation rapide

In [ ]:
# Évaluation sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Précision sur l'ensemble de test: {test_accuracy:.4f}")

704/704 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - accuracy: 0.7166 - loss: 1.3524
Précision sur l'ensemble de test: 0.7151
